In [1]:
import os
from Bio import SeqIO
import shutil
import subprocess
import webbrowser


# Converts files from abi to fasta
baseDIR = os.getcwd()
homeDIR = os.getcwd()
sourcefiles = os.listdir(baseDIR)
destinationpath = os.path.join(baseDIR, "FASTA")

def abi2Fasta(folders):
    
    for homeDIR in folders:
        counter = 0
        
        for filename in os.listdir(homeDIR):
            
            if filename.endswith(".ab1"):
                # print(filename)
                SeqIO.convert(os.path.join(homeDIR, filename), "abi", os.path.join(homeDIR, f"{filename}.fasta"), "fasta")
                counter+=1
        print (f"{counter} files were converted")

        # Moves fasta files to created FASTA folder
        destinationpath = os.path.join(homeDIR, "FASTA")
        # print(destinationpath)
        if not os.path.exists(destinationpath):
            os.makedirs(destinationpath)

        for file in os.listdir(homeDIR):
            # print(file)
            if file.endswith('.fasta'):
                shutil.move(os.path.join(homeDIR,file), os.path.join(destinationpath,file))
        print("\nFASTA files moved to FASTA folder")

In [8]:
for filename in os.listdir(homeDIR):

    if filename.endswith(".ab1"):
        # print(filename)
        SeqIO.convert(os.path.join(homeDIR, filename), "abi", os.path.join(homeDIR, f"{filename}.fasta"), "fasta")
#         counter+=1
#     print (f"{counter} files were converted")

    # Moves fasta files to created FASTA folder
    destinationpath = os.path.join(homeDIR, "FASTA")
    # print(destinationpath)
    if not os.path.exists(destinationpath):
        os.makedirs(destinationpath)

    for file in os.listdir(homeDIR):
        # print(file)
        if file.endswith('.fasta'):
            shutil.move(os.path.join(homeDIR,file), os.path.join(destinationpath,file))
    print("\nFASTA files moved to FASTA folder")


FASTA files moved to FASTA folder

FASTA files moved to FASTA folder

FASTA files moved to FASTA folder

FASTA files moved to FASTA folder


In [9]:
from Bio import SeqIO
record = SeqIO.parse(f"A_forward.ab1", "abi")
print(f"Starting with {record.id}")
count = SeqIO.write(record, "A_forward.fastq", "fastq")

In [ ]:
seqHandle = SeqIO.parse("file.ab1", "abi")

for seq in seqHandle:
    seqName = seq.id
    seqStr = str(seq.seq)
    seqQual = seq.letter_annotations["phred_quality"]

In [12]:
for record in SeqIO.parse("A_forward.fastq", "fastq"):
    print("%s %s" % (record.id, record.seq))
    print(record)
    print(record.letter_annotations["phred_quality"])

A TCGCCGCMTMGCWCTATGGAGGAGCCGCAGTMGATCCTAGCGTCGAGCCCCCTCTGAGTCAGGAAACATTTTCAGACCTATGGAAACTACTTCCTGAAAACAACGTTCTGTCCCCCTTGCCGTCCCAAGCAATGGATGATTTGATGCTGTCCCCGGACGATATTGAACAATGGTTCACTGAAGACCCAGGTCCAGATGAAGCTCCCAGAATGCCAGAGGCTGCTCCCCGCGTGGCCCCTGCACCAGCAGCTCCTACACCGGCGGCCCCTGCACCAGCCCCCTCCTGGCCCCTGTCATCTTCTGTCCCTTCCCAGAAAACCTACCAGGGCAGCTACGGTTTCCGTCTGGGCTTCTTGCATTCTGGGACAGCCAAGTCTGTGACTTGCACGTACTCCCCTGCCCTCAACAAGATGTTTTGCCAACTGGCCAAGACCTGCCCTGTGCAGCTGTGGGTTGATTCCACACCCCCGCCCGGCACCCGCGTCCGCGCCATGGCCATCTACAAGCAGTCACAGCACATGACGGAGGTTGTGAGGCGCTGCCCCCACCATGAGCGCTGCTCAGATAGCGATGGTCTGGCCCCTCCTCAGCATCTTATCCGAGTGGAAGGAAATTTGCGTGTGGAGTATTTGGATGACAGAAACACTTTTCGACATAGTGTGGTGGTGCCCTATGAGCCGCCTGAGGTTGGCTCTGACTGTACCACCATCCACTACAACTACATGTGTACAGTTCCTGCATGGGCGGCATGAACCGGAGGCCCATCCCTCMCCATCATCMCMMCTGGTARRACYTCCMRGTGGTTWAAT
ID: A
Name: A
Description: A
Number of features: 0
Per letter annotation for: phred_quality
Seq('TCGCCGCMTMGCWCTATGGAGGAGCCGCAGTMGATCCTAGCGTCGAGCCCCCTC...AAT')
[7, 5, 5, 5, 5, 5, 5, 1, 13,

In [76]:
print('Trimming reads below phred score of 20')
seq = record.letter_annotations["phred_quality"]
window_size = 3
selected={}
for i in range(len(seq) - window_size + 1):
    current_window = seq[i: i + window_size]
    if sum(current_window)/window_size >= 20:
        selected[current_window[-1]] = i+window_size-1
#         print(record.seq[current_window[-1]])
        break
for i in reversed(range(len(seq) - window_size + 1)):
    current_window = seq[i: i + window_size]
    if sum(current_window)/window_size >= 20:
        selected[current_window[-1]] = i+window_size-1
#         print(record.seq[current_window[-1]])
        break
selected_list = list(selected.keys())
selected_seq = record.seq[selected_list[0]:len(record.seq)-selected_list[1]-1]

In [77]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

In [80]:
result = NCBIWWW.qblast(program="blastn",
                        database="nr",
                        sequence=selected_seq,
                        ncbi_gi=True,
                        format_type="XML",
                        hitlist_size=10,
                        genetic_code=1,
                      entrez_query="txid5654[ORGN]")

TypeError: '_io.StringIO' object is not callable

In [81]:
save_xml = open("result.xml", "w")
save_xml.write(result.read())
save_xml.close()

In [92]:
save_xml = open("result.xml", "r")
blast_records = NCBIXML.parse(save_xml)

with open(f"{record.id}.tsv", "w") as opened_tsv:
    opened_tsv.write("hit_name\thit_length\talignment_score\tgaps\te-value\n")
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                print('****Alignment****')
                print('sequence:', alignment.title)
                print('length:', alignment.length)
                print('score:', hsp.score)
                print('gaps:', hsp.gaps)
                print('e-value:', hsp.expect)
                print(hsp.query)
                print(hsp.match)
                print(hsp.sbjct)
                opened_tsv.write(f"{alignment.title}\t{alignment.length}\t{hsp.score}\t{hsp.gaps}\t\
                                {hsp.query}\t{hsp.match}\t{hsp.sbjct}\n")
save_xml.close()

****Alignment****
sequence: gi|1002449808|ref|XM_015801976.1| Leptomonas pyrrhocoris hypothetical protein mRNA
length: 2083
score: 47.0
gaps: 0
e-value: 0.10991
CACCGGCGGCCCCTGCACCAGCCCCC
|||||||||||||||| |||||||||
CACCGGCGGCCCCTGCTCCAGCCCCC
****Alignment****
sequence: gi|1002458271|ref|XM_015797676.1| Leptomonas pyrrhocoris hypothetical protein mRNA
length: 1935
score: 45.0
gaps: 1
e-value: 0.383622
CCGCGTGGCCCCTGCACCAGCAGCTCCTACAC
||||| |||| |||||||||||||||| ||||
CCGCG-GGCCGCTGCACCAGCAGCTCCGACAC
****Alignment****
sequence: gi|1371035981|gb|CP027838.1| Leishmania infantum strain TR01 isolate Lin_TR01 chromosome 5, complete sequence
length: 450635
score: 41.0
gaps: 0
e-value: 4.67348
TCCCCGCGTGGCCCCTGCACCAG
|||||||||||||||||| ||||
TCCCCGCGTGGCCCCTGCGCCAG
****Alignment****
sequence: gi|1371035981|gb|CP027838.1| Leishmania infantum strain TR01 isolate Lin_TR01 chromosome 5, complete sequence
length: 450635
score: 41.0
gaps: 0
e-value: 4.67348
TCCCCGCGTGGCCCCTGCACCAG
|||||||||||||||||| ||

In [93]:
os.getcwd()

'C:\\Users\\DELL\\Downloads\\ITSAC'

In [1]:
import sys
import os
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

In [4]:
input_path = os.getcwd()

file_list = []
for file in os.listdir(input_path):
    if file.endswith(".ab1"):
        print(os.path.join(input_path, file))
        file_path = os.path.join(input_path, file)
        file_name = file_path.split("\\")[-1]
        name = file_name.split(".")[0]
        fastq_name = f"{name}.fastq"
        rest_of_path = file_path.rsplit("\\",1)[0]
        file_list.append(rest_of_path+"\\"+fastq_name)
        SeqIO.convert(file_name, "abi", fastq_name,"fastq")
print(file_list)

records = []
for f in file_list:
    for record in SeqIO.parse(f, "fastq"):
        records.append(record)
SeqIO.write(records, "all_seqs.fastq", 'fastq')

records_dict = {}
best_opened_tsv = open(f"best_hits_per_seq.tsv", "a") 
all_seq_fasta = open("all_seqs.fasta", 'w')
best_opened_tsv.write("sample_ID\thit_name\thit_length\talignment_score\tgaps\te-value\tquery_seq\tmatches\tsubject_seq\n")

for record in SeqIO.parse("all_seqs.fastq", "fastq"):
    print(f"Starting with {record.id}")
    print('Trimming reads below phred score of 20')
    seq = record.letter_annotations["phred_quality"]
    window_size = 3
    selected={}
    for i in range(len(seq) - window_size + 1):
        current_window = seq[i: i + window_size]
        if sum(current_window)/window_size >= 20:
            selected[current_window[-1]] = i+window_size-1
    #         print(record.seq[current_window[-1]])
            print("Number of bases chopped from 5' end =>",len(seq[:i+window_size-1]))
            break
    for i in reversed(range(len(seq) - window_size + 1)):
        current_window = seq[i: i + window_size]
        if sum(current_window)/window_size >= 20:
            selected[current_window[-1]] = i+window_size-1
    #         print(record.seq[current_window[-1]])
            print("Number of bases chopped from 3' end =>",len(seq)-len(seq[:i+window_size-1]))
            break

    selected_list = list(selected.keys())
    selected_seq = record.seq[selected_list[0]:len(record.seq)-selected_list[1]-1]
    records_dict[record.id] = str(selected_seq)

    result = NCBIWWW.qblast(program="blastn",
                        database="nr",
                        sequence=selected_seq,
                        ncbi_gi=True,
                        format_type="XML",
                        hitlist_size=10,
                        genetic_code=1,
                      entrez_query="txid5654[ORGN]")
    save_xml = open(f"{record.id}_blast_result.xml", "w")
    save_xml.write(result.read())

    save_xml = open(f"{record.id}_blast_result.xml", "r")
    blast_records = NCBIXML.parse(save_xml)

    with open(f"{record.id}.tsv", "w") as opened_tsv:
        opened_tsv.write("hit_name\thit_length\talignment_score\tgaps\te-value\tquery_seq\tmatches\tsubject_seq\n")
        counter = 0
        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    print("writing to opened_tsv")
                    opened_tsv.write(f"{alignment.title}\t{alignment.length}\t{hsp.score}\t{hsp.gaps}\t{hsp.expect}\t{hsp.query}\t{hsp.match}\t{hsp.sbjct}\n")
                    if counter == 0:
                        print("writing to best_opened_tsv")
                        best_opened_tsv.write(f"{record.id}\t{alignment.title}\t{alignment.length}\t{hsp.score}\t{hsp.gaps}\t{hsp.expect}\t{hsp.query}\t{hsp.match}\t{hsp.sbjct}\n")
                    counter+=1
                    print(counter)
        opened_tsv.close()
    save_xml.close()
best_opened_tsv.close()

print(records_dict)
for item in records_dict.items:
    all_seq_fasta.write(f">{item.key}\n{item.value}\n")
all_seq_fasta.close()
print("done")

C:\Users\DELL\Downloads\ITSAC\2020-09-22_C04_AM27H.ab1
['C:\\Users\\DELL\\Downloads\\ITSAC\\2020-09-22_C04_AM27H.fastq']
Starting with AM27H
Trimming reads below phred score of 20
Number of bases chopped from 5' end => 17
Number of bases chopped from 3' end => 1
writing to opened_tsv
writing to best_opened_tsv
1
writing to opened_tsv
2
writing to opened_tsv
3
writing to opened_tsv
4
writing to opened_tsv
5
writing to opened_tsv
6
writing to opened_tsv
7
writing to opened_tsv
8
writing to opened_tsv
9
writing to opened_tsv
10
writing to opened_tsv
11
writing to opened_tsv
12
writing to opened_tsv
13
writing to opened_tsv
14
{'AM27H': 'GGGTGATCGGCTCCTACCAAGCGATCAAGCACAAGCTTGCCGATGTCCATATCGCGATCGAGCTGGCCCGTCCGCTGGTACACGGCGCGGCATTGTCGCTGCAACCCCGCGATGTCAGCGCCGCCAAGGTCGCCGCTGGCGGCGCGGCCCGGCTGGCGGCTCGCAGCGCGTTGCAGACCCACGGTGCTATCGGGTTTACCCAGGAACACGACCTGTCGCTGTGGCTGCTGCGAGTGCAGGCGTTGCGTTCGGCGTGGGGTACACCAGAAGTACACCGGCGGCGGGTGCTCGAGGCGCTATGACCCCCGCCGGCGACGATGGGCAGCGCAGCGATGAGAAGGAGCGGCGGCGATGACCG

TypeError: 'builtin_function_or_method' object is not iterable